# RBI-QA Chatbot: Using Langchain | PineCone | Redis | OpenAi

In [1]:
import os
from dotenv import load_dotenv

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

load_dotenv()
# This is needed to enable LangSmith logging/tracing for chains, prompts, and LLM calls.
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')

# Setting this to 'true' will start recording all your chain/agent/LLM activity for debugging and evaluation.
os.environ['LANGCHAIN_TRACING_V2'] = 'true'

# This helps organize and filter traces inside the LangSmith dashboard based on this project.
os.environ['Langchain_Project'] = os.getenv('LANGCHAIN_PROJECT')

### Utilize OenAi embeddings to convert text chunks into vectors

In [2]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

C:\Users\lalra\AppData\Local\Temp\ipykernel_11260\1311270069.py:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)


### Load and Split documents into Chunks

In [3]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = PyPDFLoader('rbi-annual-report-2023-2024.pdf')
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size= 1000,
    chunk_overlap= 200,
    separators=["\n\n", "\n", ".", " ", ""]
)

text_docs = text_splitter.split_documents(documents)
# Add metadata for source type
for doc in text_docs:
    doc.metadata["source"] = "text"

text_docs[:5]

[Document(metadata={'producer': 'Adobe PDF Library 17.0', 'creator': 'Adobe InDesign 19.3 (Windows)', 'creationdate': '2024-05-29T12:25:28+05:30', 'moddate': '2024-06-03T16:37:45+05:30', 'title': '', 'trapped': '/False', 'source': 'text', 'total_pages': 310, 'page': 1, 'page_label': '2'}, page_content='Report of the Central Board of Directors on the working of the Reserve Bank of India\nfor the year ended March 31, 2024 submitted to the Central Government in terms of  \nSection 53(2) of the Reserve Bank of India Act, 1934\nRESERVE BANK OF INDIA ANNUAL REPORT\n2023-24'),
 Document(metadata={'producer': 'Adobe PDF Library 17.0', 'creator': 'Adobe InDesign 19.3 (Windows)', 'creationdate': '2024-05-29T12:25:28+05:30', 'moddate': '2024-06-03T16:37:45+05:30', 'title': '', 'trapped': '/False', 'source': 'text', 'total_pages': 310, 'page': 5, 'page_label': '6'}, page_content='GOVERNOR\nShaktikanta Das\nDEPUTY GOVERNORS\nMichael Debabrata Patra\nM. Rajeshwar Rao\nT. Rabi Sankar\nSwaminathan J.\

### Extract Tables from a PDF with pdfplumber

In [4]:
import pdfplumber
import pandas as pd

def extract_tables_from_pdf(pdf_path: str):
    all_tables = []
    
    with pdfplumber.open(pdf_path) as pdf:
        for page_num, page in enumerate(pdf.pages, start=1):
            tables = page.extract_tables()
            for table_num, table in enumerate(tables):
                if table:  # Skip empty tables
                    df = pd.DataFrame(table[1:], columns=table[0])
                    df["page"] = page_num
                    df["table_number"] = table_num + 1
                    all_tables.append(df)
    
    return all_tables  # List of DataFrames

In [5]:
extracted_tables = extract_tables_from_pdf('rbi-annual-report-2023-2024.pdf')

In [6]:
# Display first table
if extracted_tables:
    print(extracted_tables[0].head())
else:
    print("No tables found.")

     page  table_number
0       4             1


### Convert Extracted Tables to JSON Format

In [7]:
import json

def convert_tables_to_json(tables: list):
    json_tables = []

    for i, df in enumerate(tables):
        # Clean column names and remove NaNs
        df.columns = [str(col).strip() for col in df.columns]
        df = df.dropna(how='all').dropna(axis=1, how='all').fillna("")

        json_data = df.to_dict(orient="records")

        # Wrap with metadata
        json_tables.append({
            "table_id": f"table_{i+1}",
            "page": int(df.get("page", [None])[0]) if "page" in df else None,
            "data": json_data
        })

    return json_tables

In [8]:
# Convert extracted tables
json_formatted_tables = convert_tables_to_json(extracted_tables)

C:\Users\lalra\AppData\Local\Temp\ipykernel_11260\3203751871.py:11: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  json_data = df.to_dict(orient="records")
C:\Users\lalra\AppData\Local\Temp\ipykernel_11260\3203751871.py:11: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  json_data = df.to_dict(orient="records")
C:\Users\lalra\AppData\Local\Temp\ipykernel_11260\3203751871.py:11: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  json_data = df.to_dict(orient="records")
C:\Users\lalra\AppData\Local\Temp\ipykernel_11260\3203751871.py:11: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  json_data = df.to_dict(orient="records")
C:\Users\lalra\AppData\Local\Temp\ipykernel_11260\3203751871.py:11: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  json_data = df.to_dict(orient="records")
C:\Users\lalra\AppData\Local\Temp\ipykernel_11260\3203751871

### Convert Table Rows Langchain Documents

In [9]:
from langchain.schema import Document

def table_to_documents(json_tables: list):
    docs = []
    for table in json_tables:
        # Convert table data (list of dicts) back to a DataFrame
        df = pd.DataFrame(table["data"])
        
        # Convert to markdown (GPT-friendly format)
        markdown = df.to_markdown(index=False)
        doc = Document(
            page_content=f"Table ID: {table['table_id']} (Page {table['page']})\n\n{markdown}",
            metadata={
                "source": "table",
                "table_id": table["table_id"],
                "page": table["page"]
            }
        )
        docs.append(doc)
    return docs

In [10]:
table_docs = table_to_documents(json_formatted_tables)
table_docs

[Document(metadata={'source': 'table', 'table_id': 'table_1', 'page': 4}, page_content='Table ID: table_1 (Page 4)\n\n|    |   page |   table_number |\n|:---|-------:|---------------:|\n|    |      4 |              1 |'),
 Document(metadata={'source': 'table', 'table_id': 'table_2', 'page': 26}, page_content='Table ID: table_2 (Page 26)\n\n|       |   page |   table_number |\n|:------|-------:|---------------:|\n| INDIA |     26 |              1 |'),
 Document(metadata={'source': 'table', 'table_id': 'table_3', 'page': 26}, page_content='Table ID: table_3 (Page 26)\n\n|    |   page |   table_number |\n|:---|-------:|---------------:|\n| I  |     26 |              2 |'),
 Document(metadata={'source': 'table', 'table_id': 'table_4', 'page': 26}, page_content='Table ID: table_4 (Page 26)\n\n|                          |   page |   table_number |\n|:-------------------------|-------:|---------------:|\n| ASSESSMENT AND PROSPECTS |     26 |              3 |'),
 Document(metadata={'source': '

### Merge text chunks document with table documents

In [11]:
all_docs = text_docs + table_docs  # combine both text and table docs
all_docs[:5]

[Document(metadata={'producer': 'Adobe PDF Library 17.0', 'creator': 'Adobe InDesign 19.3 (Windows)', 'creationdate': '2024-05-29T12:25:28+05:30', 'moddate': '2024-06-03T16:37:45+05:30', 'title': '', 'trapped': '/False', 'source': 'text', 'total_pages': 310, 'page': 1, 'page_label': '2'}, page_content='Report of the Central Board of Directors on the working of the Reserve Bank of India\nfor the year ended March 31, 2024 submitted to the Central Government in terms of  \nSection 53(2) of the Reserve Bank of India Act, 1934\nRESERVE BANK OF INDIA ANNUAL REPORT\n2023-24'),
 Document(metadata={'producer': 'Adobe PDF Library 17.0', 'creator': 'Adobe InDesign 19.3 (Windows)', 'creationdate': '2024-05-29T12:25:28+05:30', 'moddate': '2024-06-03T16:37:45+05:30', 'title': '', 'trapped': '/False', 'source': 'text', 'total_pages': 310, 'page': 5, 'page_label': '6'}, page_content='GOVERNOR\nShaktikanta Das\nDEPUTY GOVERNORS\nMichael Debabrata Patra\nM. Rajeshwar Rao\nT. Rabi Sankar\nSwaminathan J.\

In [12]:
metadatas = [doc.metadata for doc in all_docs]
metadatas[:1]

[{'producer': 'Adobe PDF Library 17.0',
  'creator': 'Adobe InDesign 19.3 (Windows)',
  'creationdate': '2024-05-29T12:25:28+05:30',
  'moddate': '2024-06-03T16:37:45+05:30',
  'title': '',
  'trapped': '/False',
  'source': 'text',
  'total_pages': 310,
  'page': 1,
  'page_label': '2'}]

In [13]:
# Ensure there are no None values in metadata
def clean_metadata(metadata):
    return {key: (value if value is not None else "") for key, value in metadata.items()}

In [14]:
from langchain.vectorstores import Chroma

vector_db = Chroma(persist_directory="rbi_index", embedding_function=embeddings)

vector_db.add_texts(
    texts=[doc.page_content for doc in all_docs],
    metadatas=[clean_metadata(doc.metadata) for doc in all_docs]
)

vector_db.persist()

C:\Users\lalra\AppData\Local\Temp\ipykernel_11260\3888927287.py:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_db = Chroma(persist_directory="rbi_index", embedding_function=embeddings)
C:\Users\lalra\AppData\Local\Temp\ipykernel_11260\3888927287.py:10: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vector_db.persist()


### Prompt Engineering

In [40]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

In [41]:
from langchain.retrievers.multi_query import MultiQueryRetriever

multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=vector_db.as_retriever(),
    llm=llm,
    include_original=True
)

In [42]:
# Utility to classify whether a query should use table or text data
def is_table_query(query: str) -> bool:
    table_keywords = ["table", "figure", "amount", "data", "statistics", "reserves", "value", "year", "percentage", "increase", "decrease"]
    return any(keyword.lower() in query.lower() for keyword in table_keywords)

In [43]:
def get_retriever(query: str):
    if is_table_query(query):
        return vector_db.as_retriever(search_kwargs={"filter": {"source": "table"}})
    else:
        return vector_db.as_retriever()

In [70]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableMap

# template with memory placeholder
chat_prompt = ChatPromptTemplate.from_messages([
   ("system",  """You are an expert assistant trained on the RBI Annual Report 2023-24.
    Use the provided context from the report to answer the user's query precisely.
    - If the query refers to tabular information (e.g., figures, reserves, statistics), respond with values directly from the tables in markdown format if present.
    - For every answer, if its a table data go through the table and check for its corresponding values and give response.
    - Do not mention page and table number
    - If the information is unavailable, say: "The report does not specify this."
    """
    ),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}")
])

parser = StrOutputParser()

# wrap your llm with prompt
context_chain = RunnableMap({
    "input": lambda x: x["input"],
    "chat_history": lambda x: x.get("chat_history", []),
    "context": lambda x: "\n\n".join(
        doc.page_content for doc in multi_query_retriever.invoke(x["input"])
    )
}) | chat_prompt | llm | parser

# chain = chat_prompt | llm | parser


In [63]:
import redis
REDIS_URL = os.getenv('REDIS_HOST')
# Create a Redis client from the URL
r = redis.Redis.from_url(REDIS_URL, decode_responses=True)
try:
    # Test the connection
    r.set('foo', 'bar')
    print(r.get('foo'))  # Expected output: 'bar'
except Exception as e:
    print("Redis connection failed:", e)

bar


In [64]:
from langchain_community.chat_message_histories import RedisChatMessageHistory

# function provides Redis-based history based on session_id
def get_redis_history(session_id: str):
    return RedisChatMessageHistory(
        session_id=session_id,
        url=REDIS_URL
    )

In [65]:
runnable = RunnableWithMessageHistory(
    context_chain,
    get_redis_history,
    input_message_key="input",
    history_messages_key="chat_history"
)

In [66]:
def get_chat_response(user_input: str, session_id: str) -> str:
    """
    Invokes Langchain runnables with message history with user input and session id
    
    Args:
        user_input(str): The user's query or message
        session_id(str): Unique session identifier for tracking conversation history
    
    Returns:
        str: LLM-generated response
    """
    try:
        response = runnable.invoke(
            {"input": user_input},
            config={"configurable": {"session_id": session_id}}
        )
        return response
    except Exception as e:
        return f"An error occured {str(e)}"


In [67]:
response = get_chat_response("Whay much amount was given to Foreign Exchange", "user_454")
print(response)

The total amount in Foreign Currency Assets (FCA) for March 2023 was $634.9 billion.


In [69]:
response = get_chat_response("Give the stats for Indent and Supply of Banknotes by BRBNMPL and SPMCIL (April-March)", "user_454")
print(response)

The statistics for Indent and Supply of Banknotes by BRBNMPL and SPMCIL (April-March) are as follows:

| Parameter                                     | BRBNMPL (in million pieces) | SPMCIL (in million pieces) |
|-----------------------------------------------|-----------------------------|----------------------------|
| Indent Received                              | 16,000                      | 18,000                     |
| Supply Made                                   | 15,500                      | 17,500                     |
| Shortfall                                     | 500                         | 500                        |

(Page 155, Table 6.10)
